In [1]:
import jax

from quantum_transformers.datasets import get_imdb_dataloaders
from quantum_transformers.training import train_and_evaluate
from quantum_transformers.transformers import Transformer
from quantum_transformers.quantum_layer import get_circuit

data_dir = '/global/cfs/cdirs/m4392/salcc/data'

2023-08-23 07:55:23.369679: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-23 07:55:23.369704: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-23 07:55:23.369723: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-23 07:55:26.086257: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
for d in jax.devices():
    print(d, d.device_kind)

gpu:0 NVIDIA A100-SXM4-40GB


In [3]:
(imdb_train_dataloader, imdb_valid_dataloader), vocab, tokenizer = get_imdb_dataloaders(batch_size=32, data_dir=data_dir, max_vocab_size=20_000, max_seq_len=512)
print(f"Vocabulary size: {len(vocab)}")
first_batch = next(iter(imdb_train_dataloader))
print(first_batch[0][0])
print(' '.join(map(bytes.decode, tokenizer.detokenize(first_batch[0])[0].numpy().tolist())))

Vocabulary size: 19169
[  103   109    99    42   535    97   111  2205    17   171   112   356
   104    95  5327    97    42  1447    97    42  1345  2131   152   119
  2810   113   125  4296   667   113    42  3502  3842   102  4129  7681
   107   347   108 16219   132    42   462   108    42  9910  7915    98
   157    34   101   668   102   103   111    17   171   112   356   104
    42  5374   500   152   257   119 12278   102    95  5782  4431  1364
   102  2496   410  3234    98  3093   102    42     5 11760 17463     5
  5120    17    17    17  7461    15  6851  7952    15  8347  1711    15
   615    17    34   101   668   102   103   111    17   171   112   356
   104    42   421    11    96  1637  1188    12  2027   152 13487   103
   264  5374    52    17    57    17    98   119   265    97   125  5782
  4316  2131    98  3123    42 11401   128   202  2935   124    95  5001
   253    95  9185  4923    34   101   668   102   103   111    17   152
   112   119   865   104    

In [4]:
model = Transformer(num_tokens=len(vocab), max_seq_len=512, num_classes=2, hidden_size=6, num_heads=2, num_transformer_blocks=4, mlp_hidden_size=3,
                    quantum_attn_circuit=get_circuit(), quantum_mlp_circuit=get_circuit())
train_and_evaluate(model, imdb_train_dataloader, imdb_valid_dataloader, num_classes=2, learning_rate=0.0003, num_epochs=30)

Epoch  30/30: 100%|██████████| 781/781 [01:41<00:00,  7.71batch/s, Loss = 0.5260, AUC = 88.89%]

TOTAL TIME = 3080.57s
BEST AUC = 88.89% AT EPOCH 29
